In [15]:
import pickle
import scipy
import numpy as np
from sklearn import preprocessing, svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler, normalize
from tqdm import tqdm
import pandas as pd

In [2]:
train_pred = []
train_true = []

dev_pred = []
dev_true = []

test_pred = []
test_true = []

## Читаем данные

In [4]:
with open('../data/hyperlex_pairs_ppl.pickle', 'rb') as f:
    t = pickle.load(f)

# сделаем словарь {пара:перплексия}
ppl_dict = dict()
for elements in t.items():
    v1 = elements[0][0].split('(')[0].strip(' ')
    v2 = elements[0][1].strip(',')
    ppl = elements[1]
    ppl_dict[(v1, v2)] = ppl

In [6]:
lost_train = 0
with open('../data/hyperlex/splits/lexical/hyperlex_training_all_lexical.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]
        score_10 = splitted[5]
        if (v1, v2) in ppl_dict.keys():
            train_pred.append(ppl_dict[(v1, v2)])
            train_true.append(float(score_10))
        else:
            lost_train += 1

In [7]:
lost_dev = 0
with open('../data/hyperlex/splits/lexical/hyperlex_dev_all_lexical.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]
        score_10 = splitted[5]
        if (v1, v2) in ppl_dict.keys():
            dev_pred.append(ppl_dict[(v1, v2)])
            dev_true.append(float(score_10))
        else:
            lost_dev += 1

In [8]:
lost_test = 0
with open('../data/hyperlex/splits/lexical/hyperlex_test_all_lexical.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]
        score_10 = splitted[5]
        if (v1, v2) in ppl_dict.keys():
            test_pred.append(ppl_dict[(v1, v2)])
            test_true.append(float(score_10))
        else:
            lost_test += 1

In [9]:
print(lost_train)
print(lost_dev)
print(lost_test)

81
13
18


In [135]:
test_lexical = []
with open('../data/hyperlex/splits/lexical/hyperlex_test_all_lexical.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]

        test_lexical.append((v1, v2))


test_random = []
with open('../data/hyperlex/splits/random/hyperlex_test_all_random.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]

        test_random.append((v1, v2))


In [136]:
len((set(test_lexical) & set(test_random)))

71

In [139]:
len(test_lexical), len(test_random)

(269, 655)

## Тренируем SVM классификатор

In [10]:
# тут можно вставить очистку от выбросов

In [143]:
train_df = {'forward': [], 'backward': [],  'label': [], 'from': [], 'to': []}
test_df = {'forward': [], 'backward': [],  'label': []}


with open('../data/hyperlex/splits/lexical/hyperlex_training_all_lexical.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]
        score_10 = splitted[5]
        if (v1, v2) in ppl_dict.keys():
            train_df['forward'].append(ppl_dict[(v1, v2)])
            train_df['backward'].append(ppl_dict[(v2, v1)])
            train_df['label'].append(float(score_10))
            train_df['from'].append(v2)
            train_df['to'].append(v1)
        else:
            lost_train += 1

with open('../data/hyperlex/splits/lexical/hyperlex_test_all_lexical.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]
        score_10 = splitted[5]
        if (v1, v2) in ppl_dict.keys():
            test_df['forward'].append(ppl_dict[(v1, v2)])
            test_df['backward'].append(ppl_dict[(v2, v1)])
            test_df['label'].append(float(score_10))
        else:
            lost_test += 1

In [144]:
train = pd.DataFrame(train_df)
test = pd.DataFrame(test_df)

In [153]:
train.loc[(train['forward'] < 10) & (train['label'] < 2)]

,forward,backward,label,from,to
58,5.632378,17.312578,0.55,loser,winner
79,1.264195,24.980017,1.03,dollar,cent
114,7.609787,1.872348,1.67,eagle,hawk
164,1.751765,18.298920,1.22,face,jaw
312,8.007448,5.560601,1.32,girlfriend,boyfriend
340,6.737236,74.715446,1.67,toilet,stool
469,5.435464,13.931234,0.28,aunt,uncle
490,3.560813,19.680157,1.90,whistle,hum
502,8.647842,27.846411,0.75,innocence,guilt
606,8.412739,14.042396,0.75,pearl,diamond


In [147]:
train.loc[(train['forward'] > 5000)]

,forward,backward,label,from,to
4,8008.550293,315.885620,0.00,dinner,rhyme
25,17046.480469,2331.895996,6.00,club,bat
64,5938.268066,2342.968262,1.33,worker,integrity
86,31782.513672,35.428349,1.55,bra,strap
196,5283.479004,1171.315918,0.13,landscape,virtue
200,10299.700195,228.982880,0.92,intellect,heart
228,31502.958984,147.869232,5.83,food,rabbit
239,10217.832031,385.007843,1.07,calculus,clerk
261,7420.874512,20.883997,0.17,ball,refrigerator
412,5746.648438,87.657219,3.47,problem,caffeine


In [131]:
random_test = deepcopy(test)

In [134]:
test

,forward,backward,label
0,26.051916,38.237873,6.15
1,4501.588867,701.239014,0.13
2,14.122448,7.949615,7.95
3,296.038849,1.950968,2.73
4,29.477560,10.066352,8.72
...,...,...,...
246,34.040295,4.281762,2.18
247,1.370043,4.104309,7.60
248,1.098539,4.830306,7.95
249,2.188994,25.749052,8.62


In [122]:
train['diff'] = train['backward'] - train['forward']
train['frac'] = train['forward'] / train['backward']

test['diff'] = test['backward'] - test['forward']
test['frac'] = test['forward'] / test['backward']

In [123]:
train

,forward,backward,label,diff,frac
0,5.931981,4.968969,1.18,-0.963012,1.193805
1,30.488420,20.915428,1.37,-9.572992,1.457700
2,5468.501953,204.522263,0.83,-5263.979691,26.737930
3,308.169037,298.860352,1.80,-9.308685,1.031147
4,1.300138,383.533386,6.83,382.233248,0.003390
...,...,...,...,...,...
1688,30.611895,197.957733,0.55,167.345839,0.154639
1689,940.610779,34.310448,0.90,-906.300331,27.414704
1690,97.229393,208.553116,0.38,111.323723,0.466209
1691,20.774494,21.602917,5.92,0.828423,0.961652


In [124]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from scipy.stats import spearmanr
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from copy import deepcopy
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor


# Custom scorer function for Spearman correlation
def spearman_scorer(y_true, y_pred):
    return spearmanr(y_true, y_pred)[0]


class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop=None):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.columns_to_drop is not None:
            print(X.drop(columns=self.columns_to_drop))
            return X.drop(columns=self.columns_to_drop)
        return X

# Assuming df_train and df_test are your DataFrame variables for training and testing
# Splitting the DataFrame into features (X) and target (y)
X_train = train.drop(columns=['label'])
y_train = train['label']
X_test = test.drop(columns=['label'])
y_test = test['label']


In [129]:
import itertools

columns_to_drop = [['forward'], ['backward'], ['diff'], ['frac'], ['forward', 'backward'], ['diff', 'frac'], ['backward', 'diff', 'frac']]
normalizers = [StandardScaler(), MinMaxScaler(), Normalizer(), None]
models = [#{'model': Ridge(), 'alpha': [0.1, 0.5, 1, 10, 15]},
       #  {'model': DecisionTreeRegressor(), 'max_depth': [None, 1, 2, 3, 4 ,5, 10, 20, 30]},
        {'model': svm.SVR(), 'C': [0], 'epsilon': [0.01, 0.1,], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
     #   {'model': RandomForestRegressor(), 'n_estimators': [100], 'max_depth': [ 2, 3, 4], 'min_samples_split': [2, 5, 7]}

]

best_score = -np.inf  # Initialize with a very low score
best_params = None
best_model = None
best_drop = None


for col_drop in tqdm(columns_to_drop):
    for norm in normalizers:
        X_train = train.drop(columns=['label'])
        X_train = X_train.drop(columns=col_drop)
        X_test = test.drop(columns=['label'])
        X_test = X_test.drop(columns=col_drop)

        if norm:
            if str(norm) == 'Normalizer()':
                for column in X_train.columns:
                    X_train[column] = norm.transform(X_train[column].values.reshape(1, -1))[0]
                    X_test[column] = norm.transform(X_test[column].values.reshape(1, -1))[0]
            else:
                X_train = norm.fit_transform(X_train)
                X_test = norm.transform(X_test)

        for params in (models):
            
            product_combinations = []
            naming = []
            for k, v in params.items():
                if k == 'model':
                    continue
                
                product_combinations.append(v)
                naming.append(k)

            all_possible = itertools.product(*product_combinations)

            
            for option in (all_possible):
                cur_params = {}
                for i, n in enumerate(naming):
                    cur_params[n] = option[i]

                    model = params['model']
            # Fit the model on the training data
                    model.fit(X_train, y_train)

            # Predict on the test set
                    y_pred = model.predict(X_test)
            
            # Calculate Spearman correlation for the current model
                    current_score = spearman_scorer(y_test, y_pred)
                    
                    # Update the best score, params, and model if current model is better
                    if current_score > best_score:
                        best_drop = col_drop
                        best_score = current_score
                        best_params = cur_params
                        best_model = model

print(f"Best parameters: {best_params}")
print(f"Best Spearman correlation on test set: {best_score}")
print(f"Best model: {model}")


  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [01:25<00:00, 12.27s/it]

Best parameters: {'C': 0, 'epsilon': 0.01, 'kernel': 'linear'}
Best Spearman correlation on test set: 0.6126019164163569
Best model: SVR()


In [127]:
best_drop

['backward']

In [106]:
#rf {'n_estimators': 50, 'max_depth': 3, 'min_samples_split': 5} 0.7165725548766279
# 0.7212313939520568

array([[1.31078838e-03, 5.22907069e-01, 1.08218358e-04],
       [9.57059858e-05, 5.22232689e-01, 3.14719142e-03],
       [9.67586520e-04, 5.22792798e-01, 2.54342250e-05],
       ...,
       [1.20321495e-03, 5.22889702e-01, 5.22887408e-05],
       [9.67118031e-05, 5.21706101e-01, 1.40503186e-02],
       [2.36455344e-02, 5.33560407e-01, 1.77541027e-05]])

In [34]:
pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('scaler', 'passthrough',
                                                  ['forward', 'backward',
                                                   'diff', 'frac'])])),
                ('model', SVR(epsilon=0.2))])

In [11]:
n_samples, n_features = len(train_pred), 1
y = np.array(train_true)
X = np.array(train_pred).reshape(-1, 1)


In [13]:
X, y

(array([[ 8.87553501],
        [21.44285583],
        [ 1.5591203 ],
        ...,
        [ 3.95151901],
        [96.59487152],
        [26.34075165]]),
 array([7.5 , 0.15, 9.77, ..., 2.37, 3.  , 9.55]))

In [ ]:
regr = make_pipeline(StandardScaler(), svm.SVR(C=1.0, epsilon=0.2))
regr.fit(X, y)

In [13]:
regr.score(np.array(test_pred).reshape(-1, 1), np.array(test_true))

0.11643634884762444

In [15]:
svm_test_pred = regr.predict(np.array(test_pred).reshape(-1, 1))
print('Pearson: ', scipy.stats.pearsonr(svm_test_pred, test_true))
print('Spearman: ', scipy.stats.spearmanr(svm_test_pred, test_true))

Pearson:  PearsonRResult(statistic=0.5215759959640544, pvalue=6.513973719853692e-19)
Spearman:  SignificanceResult(statistic=0.7058573767815767, pvalue=3.6753305791994724e-39)


Теперь будем экспериментировать с параметрами

In [10]:
kernels = ['linear', 'rbf', 'sigmoid'] # 'poly', 'precomputed'
scalers = [StandardScaler(), MaxAbsScaler(), MinMaxScaler(), RobustScaler()]

with open('svm_lexical_split_experiments.csv', 'w', encoding='utf-8') as f:
    f.write('kernel,scaler,pearson,spearman\n')
    for kernel in kernels:
        for si, scaler in tqdm(enumerate(scalers)):
            y = np.array(train_true)
            X = np.array(train_pred).reshape(-1, 1)
            regr = make_pipeline(scaler, svm.SVR(C=1.0, epsilon=0.2, kernel=kernel))
            regr.fit(X, y)
            svm_test_pred = regr.predict(np.array(test_pred).reshape(-1, 1))
            pearson = scipy.stats.pearsonr(svm_test_pred, test_true)
            spearman = scipy.stats.spearmanr(svm_test_pred, test_true)
            f.write(kernel+','+str(si)+','+str(pearson[0])+','+str(spearman[0])+'\n')

4it [00:00,  5.25it/s]
4it [00:00, 10.37it/s]
4it [00:00, 20.91it/s]


In [154]:
test

,forward,backward,label
0,26.051916,38.237873,6.15
1,4501.588867,701.239014,0.13
2,14.122448,7.949615,7.95
3,296.038849,1.950968,2.73
4,29.477560,10.066352,8.72
...,...,...,...
246,34.040295,4.281762,2.18
247,1.370043,4.104309,7.60
248,1.098539,4.830306,7.95
249,2.188994,25.749052,8.62


In [195]:
y = np.array(test['label'])

norm_test_pred = normalize([np.array(test['forward'])], norm='max')[0].reshape(-1, 1).flatten()

In [196]:
print('Spearman: ', scipy.stats.spearmanr(norm_test_pred, y))

Spearman:  SignificanceResult(statistic=-0.7058839380077765, pvalue=3.6411575820150636e-39)


In [13]:
# с нормализацией
y = np.array(train_true)
X = normalize([np.array(train_pred)])[0].reshape(-1, 1)
regr = make_pipeline(StandardScaler(), svm.SVR(C=1.0, epsilon=0.2))
regr.fit(X, y)

norm_test_pred = normalize([np.array(test_pred)])[0].reshape(-1, 1)
svm_test_pred = regr.predict(norm_test_pred)
print('Pearson: ', scipy.stats.pearsonr(svm_test_pred, test_true))
print('Spearman: ', scipy.stats.spearmanr(svm_test_pred, test_true))

Pearson:  PearsonRResult(statistic=0.6178086955415456, pvalue=8.225887918634271e-28)
Spearman:  SignificanceResult(statistic=0.7016386951688859, pvalue=1.5993761706225828e-38)
